In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans


In [2]:
# dictionary of title_Id -> title_name
fp=open("title_Id.txt",'r')

titleId_titleName={}
for line in fp:
    line=line.strip().split('#')
    #print line[0], line[1]
    if len(line)!=2:
        if len(line)>2:
            key=line[-1]
            value=""
            for i in range(len(line)-1):
                value=value+line[i]
            titleId_titleName[key]=value 
    else:
        titleId_titleName[line[1]]=line[0]

fp.close()

    

In [3]:
# dictionary of journal_Id -> journal_name
fp=open("journal_Id.txt",'r')

journalId_JournalName={}
for line in fp:
    line=line.strip().split("#")
    journalId_JournalName[line[1]]=line[0]

fp.close()

In [4]:
# dictionary of author_Id -> author_name
fp= open("author_Id.txt",'r')

authorId_authorName={}
for line in fp:
    line = line.strip().split("#")
    authorId_authorName[line[1]]=line[0]
fp.close()

In [5]:
fp= open("title_Journal_Author.txt",'r')

authorList=[]
titleJournalList=[]
for line in fp:
    line = line.strip().split("#")
    curr= titleId_titleName[line[0]] + " " + journalId_JournalName[line[1]]
    titleJournalList.append(curr)
    tempList=line[2].strip().split('|') #author ids list
    tempNameList=[] #authors namelist
    for i in tempList:
        tempNameList.append(authorId_authorName[i])
    authorList.append(tempNameList)
    
fp.close()

In [6]:
#print len(authorList)
#print len(titleJournalList)
#print authorList[2], titleJournalList[2]


In [7]:
# to find a no. of unique words
vocab={}
for line in titleJournalList:
    wordList=line.strip().split()
    for word in wordList:
        if word in vocab:
            vocab[word]=vocab[word]+1
        else:
            vocab[word]=1

print len(vocab)

125296


In [8]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 70000)
vec=vectorizer.fit(titleJournalList)
vectorized=vec.transform(titleJournalList)



In [18]:
print vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=70000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [16]:
print type(vectorized)

<class 'scipy.sparse.csr.csr_matrix'>


In [10]:
km = KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1)
km.fit(vectorized)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=20, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [12]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(20):
    print "Cluster %d:" % i,
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind],
    print

Top terms per cluster:
Cluster 0:  models  graphical  comput  based  physics  using  logic  automatica  petri  nets
Cluster 1:  synthese  theory  logic  epistemic  knowledge  problem  science  explanation  truth  scientific
Cluster 2:  based  syst  knowl  knowledge  using  systems  fuzzy  approach  model  data
Cluster 3:  comput  physics  based  journal  computer  reliability  computing  logic  using  networks
Cluster 4:  generation  future  comp  syst  based  grid  data  parallel  computing  distributed
Cluster 5:  fuzzy  systems  intelligent  journal  based  using  decision  control  making  method
Cluster 6:  web  agent  intelligence  systems  based  multi  information  agents  social  using
Cluster 7:  safety  eng  amp  sys  rel  analysis  systems  risk  based  model
Cluster 8:  acm  trans  secur  inf  syst  based  control  access  systems  secure
Cluster 9:  appl  eng  ai  commun  algebra  comput  based  using  fuzzy  algorithm
Cluster 10:  safety  reliability  eng  amp  sys  rel 

In [11]:
authorClusters={} # Cluster Id to List of authorName
for i in range(len(titleJournalList)):
    currList=[]
    currList.append(titleJournalList[i])
    data_features=vec.transform(currList)
    labels=km.predict(data_features)
    
    clusterId=labels[0]
    curAuthorsList=authorList[i] #authors list for current article
    if clusterId in authorClusters.keys():
        #fetch the existing list of authors for the clusterId
        tempList=authorClusters[clusterId]
        for j in curAuthorsList:
            tempList.append(j)
            
        #update authorsClusters with updated authors list
        authorClusters[clusterId]=tempList
        
    else: #create a new key with the new cluster id and make value list of authors for the current article
        authorClusters[clusterId]=curAuthorsList

# print clusters Id-> author ids



In [13]:
fp=open("Kmeans_countVectorizer_output","wr")

for i in authorClusters.keys():
    outstr=str(i) + " => " + str(authorClusters[i])
    fp.write(outstr+"\n")
    fp.write("**************************************************************************************************************************"+"\n")
fp.close()


NameError: name 'authorClusters' is not defined